## Settings

In [ ]:
from pathlib import Path

audio_folder = Path("/Users/hudsonmendes/Workspaces/hudsonmendes-estudos/cm3065-isp/exercise-04/files/Ex4_audio_files")

## Dependencies

In [ ]:
 %load_ext autoreload
 %autoreload 2

In [ ]:
from typing import Dict, List

import os

## Transcribers

In [ ]:
from asr.transcriber import Transcriber
transcriber_en = Transcriber(locale="en_US")

In [ ]:
def collect_files_per_locale(folder: Path) -> Dict[str, List[Path]]:
    files_per_locale = {}
    for root, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.endswith('.wav'):
                filepath = os.path.join(root, filename)
                locale = filepath.split('/')[-2].lower().strip()
                files_per_locale.setdefault(locale, []).append(Path(filepath))
    return files_per_locale

In [ ]:
files_per_locale = collect_files_per_locale(folder=audio_folder)
files_per_locale

In [ ]:
transcriber_en.transcribe(files_per_locale['en'][0])